<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Unsupervised_Similar_Images_Finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**I**n this project we are going to learn about how to fins similar images of and select the best one based on it's size(resolution) and store it in the main folder and send the other copies to the rubish folder

In [1]:
import os
import cv2
import glob
import tqdm
import shutil
import skimage
import IPython
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

!pip install patool
import patoolib

IPython.display.clear_output()

Step 1: Read the zip file from the drive and unzip it

In [2]:
patoolib.extract_archive("/content/drive/MyDrive/reddit_files.rar", outdir="/content/data")

Path = './data/images/'
os.rename(f"./data/{os.listdir('./data')[0]}", Path)

IPython.display.clear_output()

data = os.listdir(Path)
len(data)

2009

Step 2: create a path for the videos and rubbish data and store the video clips in the video folder and then select the images andsave it as data variable

In [3]:
Path = './data/images/'
rubish_path = './Rubish/'
data = glob.glob(f'{Path}/*g')
videos = glob.glob(f'{Path}/*.mp4')
os.makedirs(f'{Path}/Videos/', exist_ok=True)
os.makedirs(rubish_path, exist_ok=True)
for i, video_path in enumerate(videos):
    os.replace(video_path, f'{Path}/Videos/{i}.mp4')

Step 3:
create a Mean-Pooling layer

Reason:
an image is a big matrix and since we only need to compare the images from some of their pixels, we need to at first convert 3d image to a 1-dim image and then get the mean part of each block of the image

In [4]:
def avg_pooling(img, pad_size=20):
  image = abs(img[:, :, 0] - img[:, :, 1] - img[:, :, 2])
  max_pool_img = skimage.measure.block_reduce(image, (pad_size,pad_size), np.mean)
  return max_pool_img

Step 4:
we have to read the images and since we are using min-pooling, we have to resize them in a same size, for example:(1000, 1000) and then pass it through avg-pooling then flatten the image, so we have less values per image

In [5]:
arr = []
height_arr, weight_arr = [], []
for d in tqdm.tqdm(data):
  try:
      img = cv2.imread(d)
      height_arr.append(img.shape[0])
      weight_arr.append(img.shape[1])
      img = cv2.resize(img, (1000, 1000))
      img = avg_pooling(img, pad_size=50)
      img = np.ravel(img)
      arr.append(img)
  except:
    print(d)

print('\n')
print('Height Mean:',np.mean(height_arr), 'Height Min:',np.min(height_arr), 'Height Max:',np.max(height_arr))
print('Weight Mean:',np.mean(weight_arr), 'Weight Min:',np.min(weight_arr), 'Weight Max:',np.max(weight_arr))

100%|██████████| 1970/1970 [01:29<00:00, 22.00it/s]



Height Mean: 1343.7598984771573 Height Min: 194 Height Max: 8598
Weight Mean: 1111.6837563451777 Weight Min: 149 Weight Max: 6144


Step 5:
In this step we go through the images that we stored in arr and get the average of the difference of each image values from every other images in the directory, and only select the ones that have mean average error < 15

at last we sort the selected images in temp_data and sort it by temp_size because we want the images with higher resolution(bigger sizes) and store the sorted image pathes in to mae_err

In [ ]:
mae_arr = []
for i, A in tqdm.tqdm(enumerate(arr)):
  temp_data = []
  temp_size = []
  mean_mae_arr = [(abs(A - b)).mean() for b in arr]
  for j, mae in enumerate(mean_mae_arr):
    if mae < 15:
      img = cv2.imread(data[j])
      img_size = img.shape[0] * img.shape[1]
      temp_data.append(data[j])
      temp_size.append(img_size)

  temp_data = np.array(temp_data)[np.argsort(temp_size)[::-1]]
  mae_arr.append(temp_data)

mae_arr = np.array(mae_arr)
mae_arr.shape

Step 6:
In the last step, we store the first images of each path_list and get rid of the othe copies

In [7]:
for id, path_list in tqdm.tqdm(enumerate(mae_arr)):
  for n, p in enumerate(path_list):
    if os.path.exists(p):
      if n==0:
        os.replace(p, f'{Path}/img_{id}.jpg')
      else:
        os.replace(p, f'{rubish_path}/img_{id}_{n}.jpg')

1970it [00:00, 18019.87it/s]


#test 1

In [ ]:
mae_arr = []
for i, A in tqdm.tqdm(enumerate(arr)):
  temp = []
  mean_mae_arr = [(abs(A - b)).mean() for b in arr]
  for j, mae in enumerate(mean_mae_arr):
    if mae < 15:
      img = cv2.imread(p)
      img_size = img.shape[0] * img.shape[1]
      temp.append(data[j])
  mae_arr.append(temp)

mae_arr = np.array(mae_arr)
mae_arr.shape

In [ ]:
new_data = glob.glob('/content/data/images/*g')
for i in new_data:
  p1, p2, p3 = i.split('_')
  if '0.jpg' == p3:
    os.replace(i, f'./data/images/Images/img_{p2}.jpg')

In [8]:
# for id, path_list in tqdm.tqdm(enumerate(mae_arr)):
#   best_img, best_size = path_list[0], 0
#   for n, p in enumerate(path_list):
#       if os.path.exists(p):
#           img = cv2.imread(p)
#           img_size = img.shape[0] * img.shape[1]
#           if img_size >= best_size:
#               best_img  = p
#               best_size = img_size
#           else:
#               os.replace(p, f'./Rubish/img_{id}.jpg')
#   if os.path.exists(p):
#       os.replace(best_img, f'{Path}/Images/img_{id}.jpg')

1970it [00:00, 26063.74it/s]


In [ ]:
# mae_arr = []
# arr2 = []
# arr3 = []
# for i, A in tqdm.tqdm(enumerate(arr)):
#   temp = []
#   mean_mae_arr = np.array([(abs(A - b)).mean() for b in arr])
#   # mean_mae_arr = categorize(mean_mae_arr)
#   mean_mae = np.mean(mean_mae_arr)
#   arr2.append(mean_mae_arr)
#   arr3.append(mean_mae)
#   for j, mae in enumerate(arr):
#     temp.append(data[j])
#   mae_arr.append(temp)

# arr2 = np.array(arr2)

# temp_arr = []
# score_arr= []
# for score, path in zip(arr2, mae_arr):
#   if len(np.unique(score)) > 2:
#     t1, t2 = [], []
#     for s, p in zip(score, path):
#       if s < 50:
#         t1.append(s)
#         t2.append(p)
#     score_arr.append(t1)
#     temp_arr.append(t2)

# len(temp_arr), len(arr2)

In [ ]:
# N=0
# N+=1

# # print(N, arr3[N], np.max(arr2[N]), np.unique(arr2[N], return_counts=True))
# mae_pathes = np.array(mae_arr[N])[np.argsort(arr2[N])]
# mae_scores = np.array(arr2[N])[np.argsort(arr2[N])]
# # print(N, list(np.round(mae_scores[:5], 2)))
# fig = plt.figure(figsize=(5, 5))
# for i in range(20):
#   img = plt.imread(mae_pathes[i])
#   img = cv2.resize(img, (100, 100))
#   fig.add_subplot(4, 5, i+1)
#   plt.gca().set_title(round(mae_scores[i], 2))
#   plt.grid(False)

#   plt.axis('off')
#   plt.imshow(img)

In [ ]:
for id, path_list in tqdm.tqdm(enumerate(mae_arr)):
  best_img, best_size = path_list[0], 0
  for n, p in enumerate(path_list):
      if os.path.exists(p):
          img = cv2.imread(p)
          img_size = img.shape[0] * img.shape[1]
          if img_size >= best_size:
              best_img  = p
              best_size = img_size
          else:
              os.replace(p, f'./Rubish/img_{id}.jpg')
  if os.path.exists(p):
      os.replace(best_img, f'{Path}/Images/img_{id}.jpg')

1970it [01:50, 17.76it/s]


In [ ]:
# for id, path_list in tqdm.tqdm(enumerate(mae_arr)):
#   if len(path_list) > 1:
#     best_img, best_size = path_list[0], 0
#     for n, p in enumerate(path_list):
#         if os.path.exists(p):
#             img = cv2.imread(p)
#             img_size = img.shape[0] * img.shape[1]
#             if img_size > best_size:
#                 best_img  = p
#                 best_size = img_size
#             else:
#                 os.replace(p, f'./Rubish/img_{id}.jpg')
#     if os.path.exists(p):
#         os.replace(best_img, f'{Path}/Images/img_{id}.jpg')
#   else:
#      p = path_list[0]
#      if os.path.exists(p):
#         os.replace(p, f'{Path}/Images/img_{id}.jpg')

1970it [01:22, 23.78it/s]


In [ ]:
# mae_arr = []
# for i, A in tqdm.tqdm(enumerate(arr)):
#   mean_mae_arr = np.array([(abs(A - b)).mean() for b in arr])
#   mae_arr.append(mean_mae_arr)

# mae_arr = np.array(mae_arr)

1970it [00:37, 53.15it/s]


In [18]:
print(len(os.listdir('/content/data/images/')))
print(len(os.listdir('/content/Rubish')))
print(len(os.listdir('/content/data/images/Images')))
print(len(os.listdir('/content/data/images/Videos')))
708+317+914+39

708
317
914
39


1978

In [ ]:
# os.listdir('/content/data/images/')

In [ ]:
def categorize(arr):
  arr[arr <= 10] = 0
  # arr[np.logical_and(arr > 1 , arr <= 10)] = 10
  arr[np.logical_and(arr > 10, arr <= 20)] = 20
  arr[np.logical_and(arr > 20, arr <= 30)] = 30
  arr[np.logical_and(arr > 30, arr <= 40)] = 40
  arr[arr > 40] = 100
  return arr

In [ ]:
mae_arr = []
arr2 = []
arr3 = []
for i, A in tqdm.tqdm(enumerate(arr)):
  temp = []
  mean_mae_arr = np.array([(abs(A - b)).mean() for b in arr])
  # mean_mae_arr = categorize(mean_mae_arr)
  mean_mae = np.mean(mean_mae_arr)
  arr2.append(mean_mae_arr)
  arr3.append(mean_mae)
  for j, mae in enumerate(arr):
    temp.append(data[j])
  mae_arr.append(temp)

arr2 = np.array(arr2)


1970it [00:38, 51.29it/s]


In [ ]:
arr_cat = []
for a in arr2:
    arr_cat.append(categorize(a))

arr_cat[:10]

[array([  0., 100., 100., ..., 100., 100., 100.]),
 array([100.,   0., 100., ..., 100., 100., 100.]),
 array([100., 100.,   0., ..., 100., 100., 100.]),
 array([100., 100., 100., ..., 100., 100., 100.]),
 array([100., 100., 100., ..., 100., 100., 100.]),
 array([100., 100., 100., ..., 100., 100., 100.]),
 array([100., 100., 100., ..., 100., 100., 100.]),
 array([100., 100., 100., ..., 100., 100., 100.]),
 array([100., 100., 100., ..., 100., 100., 100.]),
 array([100., 100., 100., ..., 100., 100., 100.])]

In [ ]:
temp_arr = []
score_arr= []
for score, path in zip(arr2, mae_arr):
  if len(np.unique(score)) > 2:
    t1, t2 = [], []
    for s, p in zip(score, path):
      if s < 50:
        t1.append(s)
        t2.append(p)
    score_arr.append(t1)
    temp_arr.append(t2)

len(temp_arr), len(arr2)

(759, 1970)

In [ ]:
p=0
p+=1
N = ss[p]

mae_pathes = np.array(temp_arr[N])[np.argsort(score_arr[N])]
mae_scores = np.array(score_arr[N])[np.argsort(score_arr[N])]
print(N, list(np.round(mae_scores[:5], 2)))
fig = plt.figure(figsize=(5, 5))
for i in range(min(len(mae_scores), 20)):
  img = plt.imread(mae_pathes[i])
  img = cv2.resize(img, (100, 100))
  fig.add_subplot(4, 5, i+1)
  plt.gca().set_title(round(mae_scores[i], 2))
  plt.grid(False)
  plt.axis('off')
  plt.imshow(img)

In [ ]:
# N=0
N+=1
# 10 [0.0, 40.0 | 100.0, 100.0, 100.0]
# 18 [0.0, 0.0, 10.0, 10.0 | 100.0]
# 27 [0.0, 27.0, 28.45, 34.09 | 41]
# 28 [0.0, 3.69| 47.34]
# 29 [0.0, 0.77, 9.37, 9.38, 13.45, 13.55 | 52]
# 31 [0.0, 0.0, 0.13, 0.13| 33.55]
# 33 [0.0, 0.9, 3.21, 3.3, 3.75, 3.9 | 49]
# 38 [0.0, 0.93 | 28.42]
# 53 [0.0, 8.37, 16.78, 17.0 | 53.69]
# 58 [0.0, 0.0, 0.13, 0.13 | 33.54]
# 59 [0.0, 21.51, 27.0, 31.85 | 46.31]
# 61 [0.0, 1.35, 1.77, 1.87, 37.57, 37.87, 49.9]
# 88 [0.0, 10.0, 30.0, 30.0 | 100.0]
# 89 [0.0, 0.0 | 20.0, 30.0, 30.0]
# 94 [0.0, 10.0 | 100.0, 100.0, 100.0]
# 99 [0.0, 10.0, 40.0, 40.0 | 100.0]


print(N, arr3[N], np.max(arr2[N]), np.unique(arr2[N], return_counts=True))
mae_pathes = np.array(mae_arr[N])[np.argsort(arr2[N])]
mae_scores = np.array(arr2[N])[np.argsort(arr2[N])]
print(N, list(np.round(mae_scores[:5], 2)))
fig = plt.figure(figsize=(5, 5))
for i in range(20):
  img = plt.imread(mae_pathes[i])
  img = cv2.resize(img, (100, 100))
  fig.add_subplot(4, 5, i+1)
  plt.gca().set_title(round(mae_scores[i], 2))
  plt.grid(False)
  plt.axis('off')
  plt.imshow(img)

In [ ]:
plt.figure(figsize=(2^16, 2^16))
plt.imshow(arr2)
plt.savefig('heatmap.png')
plt.savefig('heatmap.pdf')

In [ ]:
for id, path_list in tqdm.tqdm(enumerate(mae_arr)):
  if len(path_list) > 1:
    best_img, best_size= path_list[0], 0
    for n, p in enumerate(path_list):
        if os.path.exists(p):
            img = cv2.imread(p)
            img_size = img.shape[0] * img.shape[1]
            if img_size > best_size:
                best_img  = p
                best_size = img_size
    if os.path.exists(p):
        os.replace(best_img, f'{Path}/Images/img_{id}.jpg')
  else:
     p = path_list[0]
     if os.path.exists(p):
        os.replace(p, f'{Path}/Images/img_{id}.jpg')